# Linux

## Administrar Usuarios en Linux

* crear usuarios tstark, cdanvers, dprince en servidor
* crear grupo superhero
* hacer usuario tstark  un superusuario
* agregar grupo uperhero a los usuarios tstark, cdanvers, dprince
* bloquear cuenta de usuario dprince


In [ ]:
# abrir consola ssh Linux 

# entrar con usuario ssh
# usuario: cloud_user
# pass: pass
# usuario: cloud_user
# Public IP address of the Server1

ssh usuario@Public IP
password: pass# contraseña


In [ ]:
# agregar usuarios

useradd tstark
useradd dprince
useradd cdanvers 

In [ ]:
# crear grupo

groupadd superhero

In [ ]:
# cambiar grupo wheel a usuario tstark
usermod -g wheel tstark # id usuario

# agregar grupo superhero como grupo suplementario para todos los usuarios
usermod -aG grupo usuario

usermod -aG superhero tstark
usermod -aG superhero dprince
usermod -aG superhero cdanvers

# cada usuario tiene su propio grupo y pertenece al grupo superhero


In [ ]:
# bloquear usuario dprince
usermod -L dprince

## Crear usuarios sudo

* crear 2 usuarios 
* cambiar archivo /etc/sudoers
* probar accesos
* set administrador web 

In [ ]:
# crear usuario sudo
sudo useradd -m gfreeman

# agregar usuario avance
# asignar usuario avance a  grupo wheel suplementario
sudo useradd -G wheel -m avance

# set contraseña de los usuarios:  pass: LASudo321

sudo passwd gfreeman
pass: LASudo321

sudo passwd avance
pass: LASudo321


In [ ]:
# comprobar archivos /etc/sudoers permiten al grupo wheel acceso a los comandos sudo

sudo visudo

In [ ]:
# cambiar a cuenta avance, - para usar login de shell
sudo su - avance

# salir de la cuenta sudo
exit

In [ ]:
# configurar usuario gfreeman para webmaster
# crear nuevo archivo  sudoers en diccionario /etc/sudoers.d con entrada unica para webmasters

sudo visudo -f /etc/sudoers.d/web_admin
Cmnd_Alias  WEB = /bin/systemctl restart httpd.service, 
/bin/systemctl reload httpd.service

# permitir acceso a sudo a usuario gfreeman     
gfreeman ALL=WEB



In [ ]:
# entrar en cuenta gfreeman
sudo su - gfreeman

# intentar reiniciar 
sudo systemctl restart httpd.service

## Uso de Terminal SSH, redireccionamiento y Permisos de Linux

* permitir ssh sin pass de dev@server1 a dev@server2
* copiar todos los archivos .tar de  server1 a server2
* set umask para que los nuevos archivos sean read/write por su propietario solamente
* run el script 'deploy'


In [ ]:
# entrar a ssh con usuario dev y Public IP address
ssh usuario@Public IP address
pass: password servidor1


In [ ]:
# generar key
ssh-keygen

# copiar a  private IP de servidor2
ssh-copy-id  private IP servidor2

# entrar/loggin al servidor2 con IP publica
ssh Public IP servidor2
 

In [ ]:
# copiar todos los archivos .tar de servidor1 a servidor2 de  /home/dev/
scp *.gz server2_PRIVATE_IP:~/

In [ ]:
# ssh conectar a servidor2
ssh <server2_PRIVATE_IP>

# verificar que todos los archivos .tar desde servidor1 fueron copiados 
ll

In [ ]:
# extraer archivos .tar en servidor2
# >>: appending
# >: sobreescribir

tar -xvf deploy_content.tar.gz >> tar-output.log
tar -xvf deploy_script.tar.gz >> tar-output.log

# revisar directorio
ll


In [ ]:
# set unmask en servidor2 para que nuevos archivos sean ONLY READABLE and WRITEABLE por el propietario
umask 0066

# verificar permisos en deploy.sh
# hacer script ejecutable
chmod +x deploy.sh

# run archivo deploy.sh 
./deploy.sh


## Configurar el Boot Target por defecto

In [ ]:
## default target actual es en multi-user

# revisar que target actual es tipo multi-user
systemctl get-default

# cambiar default target para que el computedor arranque(boot) en desktop gráfico
sudo systemctl set-default graphical.target

# revisar que target actual es tipo graphical
systemctl get-default


## Agendar un servicio Systemd con Timer Units


In [ ]:

# entrar a ssh como usuario root
sudo su -

# check directorio actual
pwd

# ver recursos en el directorio
ls

# crear archivo timer unit 
vi web-backup.timer

# codigo de archivo timer unit

[Unit]
Description=Fire off the backup

[Timer]
OnCalendar=*-*-* 23:00:00
Persistent=true
Unit=web-backup.service

[Install]
WantedBy=multi-user.target

# salir de  archivo timer unit
### ESC
### :wq

In [ ]:
# copiar script shell en /usr/local/sbin
cp web-backup.sh /usr/local/sbin/

# copiar servicio y archivos timer en /etc/systemd/
cp web-backup.{service,timer} /etc/systemd/system/

# recargar el Systemd daemon para calcular las dependencias de servicio
systemctl daemon-reload

# habilitar servicios para correr un Boot
systemctl enable web-backup.service
systemctl enable web-backup.timer

# set permisos del archivo para ejecutar
chmod +x /usr/local/sbin/web-backup.sh

# iniciar servicios manualmente
systemctl start web-backup.timer web-backup.service

# check status de timer y servicio
systemctl status web-backup.timer
  Loaded: loaded (/etc/systemd/system/web-backup.timer; enabled; vendor preset: disabled)
   Active: active (waiting) since Mon 2023-08-14 10:33:07 EDT; 35s ago

systemctl status web-backup.service
● web-backup.service - Backup the web site, every day, or the boss will be mad
   Loaded: loaded (/etc/systemd/system/web-backup.service; enabled; vendor preset: disabled)
   Active: inactive (dead) since Mon 2023-08-14 10:33:07 EDT; 1min 17s ago
  Process: 4359 ExecStart=/usr/local/sbin/web-backup.sh (code=exited, status=0/SUCCESS)
 Main PID: 4359 (code=exited, status=0/SUCCESS)

# servidor listo para usarse


## Journal Control Utility: Administrar archivos y Troubleshooting


In [ ]:
# cambiar a cuenta root
sudo su -

# check el estado del servicio web
systemctl status httpd.service

# intentar iniciar el servicio web
systemctl start httpd.service

# check Journal
journalctl -u httpd.service

-- Logs begin at Mon 2023-08-14 10:10:57 EDT, end at Mon 2023-08-14 10:43:37 EDT. --
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Starting The Apache HTTP Server...
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal httpd[22861]: httpd: Could not open configuration fi
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service: main process exited, code
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Failed to start The Apache HTTP Server.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Unit httpd.service entered failed state.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service failed.

# revisar directorio donde la configuracion httpd debe estar
ls /etc/httpd/conf

httpd.conf.bkup  magic

# restaurar la configuracion httpd original del archivo  
mv /etc/httpd/conf/httpd.conf.bkup /etc/httpd/conf/httpd.conf

# reiniciar el servicio
systemctl restart httpd.service

# check el estado del servicio
systemctl status httpd.service

● httpd.service - The Apache HTTP Server
   Loaded: loaded (/usr/lib/systemd/system/httpd.service; disabled; vendor preset: disabled)
   Active: active (running) since Mon 2023-08-14 10:49:02 EDT; 39s ago
     Docs: man:httpd(8)
           man:apachectl(8)
 Main PID: 22879 (httpd)
   Status: "Total requests: 0; Current requests/sec: 0; Current traffic:   0 B/sec"
   CGroup: /system.slice/httpd.service
           ├─22879 /usr/sbin/httpd -DFOREGROUND
           ├─22880 /usr/sbin/httpd -DFOREGROUND
           ├─22881 /usr/sbin/httpd -DFOREGROUND
           ├─22882 /usr/sbin/httpd -DFOREGROUND
           ├─22883 /usr/sbin/httpd -DFOREGROUND
           └─22884 /usr/sbin/httpd -DFOREGROUND

# navegar a pagina web local 
elinks http://localhost

## Generar y cambiar SSH Keys para acceso seguro

## Admon SIS


In [ ]:
# ver nombre, direccionIP, alias
nslookup

# entrar a servidor (Jupiter)
ssh direccionIP

# ver /etc/httpd
cd /etc/httpd
ls

#ver conf.d/
cd conf.d/
ls

# ver nano
nano ssl.conf



In [ ]:
# ver conf.d
cd "/etc/httpd/conf.d"

# ver /public_html
cd /ul/sitios/cupitaller/public_html/
ls
